In [1]:
from PIL import Image
import os
import torch
import matplotlib.pyplot as plt

In [2]:
from patched_clip.patched_clip import CLIP_args, get_clip_embeddings

In [3]:
image_paths = ['test.jpg']
device = 'cpu'

In [4]:
# CLIP_args.clip_model_name = CLIPArgs.model_type
# CLIP_args.device = device
high_res_feature = True

# Load images into pillow images
images = [Image.open(image_path) for image_path in image_paths]

# Get CLIP embeddings
with torch.no_grad():
    descriptors = get_clip_embeddings(images, to_cpu=False, skip_center_crop=high_res_feature)
    descriptors = descriptors.to(device)

ML_LOGGER_USER is not set. This is required for online usage.
timing <load_clip_model>: 4.989E+00s
timing <clip_preprocess_images>: 2.102E-01s
timing <get_clip_embeddings>: 1.858E+00s
clip_process_image_mean_time: 2.0689631659988663


In [5]:
descriptors.shape

torch.Size([1, 768, 24, 32])